In [1]:
import numpy as np
from numba import cuda
import xgboost as xgb

import dask
from dask_cuda import LocalCUDACluster
from dask.delayed import delayed
from dask.distributed import Client, wait
import dask_cudf

import math

# local dev
#cluster = LocalCUDACluster()
#client = Client(cluster)
# in DataProc environment
#ToDo- get hostname programmatically
client = Client('test-m:8786')
client

Client Scheduler: tcp://test-m:8786 Dashboard: http://test-m:8787/status,Cluster Workers: 2 Cores: 2 Memory: 7.89 GB


# Perform the ETL

In [4]:
#taxi_df = dask_cudf.read_csv('/data/nyc_taxi/raw/2014/yellow_tripdata_2014-1*')
taxi_df = dask_cudf.read_csv('/data/nyc_taxi/raw/2014/yellow_*')

taxi_df.head().to_pandas()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0.0,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0.0,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0.0,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0.0,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0.0,8.75


In [5]:
def pretty(val):
    print("{:,}".format(val))

pretty(len(taxi_df))

165,114,361


In [6]:
def clean(df, mapper):    
    # some col-names include pre-pended space.. fix it
    tmp = {col:col.strip().lower() for col in list(df.columns)}
    df = df.rename(tmp)
    
    # drop any column without a supplied replacement
    for col in mapper:
        if col in mapper and mapper[col] == None and col in df.columns:
            df = df.drop(col)
    
    # rename according to supplied mapping
    df = df.rename(mapper)
        
    # fill all na values for non-object columns
    for col in df.columns:
        if df[col].dtype != 'object':
            df[col] = df[col].fillna(-1)
    
    return df

In [7]:
# drop list
col_map = dict.fromkeys([
    'vendor_id', 'dropoff_datetime', 'payment_type', 'surcharge', 'mta_tax',
    'tip_amount', 'tolls_amount', 'total_amount', 'store_and_fwd_flag'
])

parts = [dask.delayed(clean)(part, col_map) for part in taxi_df.to_delayed()]
taxi_df = dask_cudf.from_delayed(parts)

In [2]:
#Numba Kernel to calculate Haversine distance
@cuda.jit
def haversine_kernel(lat1, lon1, lat2, lon2, outputCol):
    iRow = cuda.grid(1)
    p = 0.017453292519943295 # Pi/180
    if iRow < outputCol.size:
        a = 0.5 - math.cos((lat2[iRow] - lat1[iRow]) * p)/2 + math.cos(lat1[iRow] * p) * \
            math.cos(lat2[iRow] * p) * (1 - math.cos((lon2[iRow] - lon1[iRow]) * p)) / 2                                 
        outputCol[iRow] = 12734 * math.asin(math.sqrt(a))
    
def haversine_distance(gdf):
    nRows = gdf.shape[0]
    blockSize = 128
    blockCount = nRows // blockSize + 1
    lat1_arr = gdf['pickup_latitude'].to_gpu_array()
    lon1_arr = gdf['pickup_longitude'].to_gpu_array()
    lat2_arr = gdf['dropoff_latitude'].to_gpu_array()
    lon2_arr = gdf['dropoff_longitude'].to_gpu_array()
                                   
    outputCol = cuda.device_array ( shape=(nRows), dtype=lat1_arr.dtype.name)
    
    haversine_kernel[(blockCount),(blockSize)](lat1_arr, lon1_arr, lat2_arr, lon2_arr, outputCol)
    gdf.add_column(name='h_distance', data = outputCol)
    return gdf

#Numba Kernel to calculate day of the week from Date
@cuda.jit
def day_of_the_week_kernel(output ,year, month, day):
    iRow = cuda.grid(1)
    if iRow < output.size:
        year[iRow] -= month[iRow] < 3
        month[iRow] = (month[iRow] + 9)%12 + 1
        output[iRow] = (year[iRow] + int(year[iRow]/4) - int(year[iRow]/100) + int(year[iRow]/400) + math.floor(2.6*month[iRow] - 0.2) + day[iRow] -1) % 7
    
def day_of_week(gdf):
    nRows = gdf.shape[0]
    blockSize = 128
    blockCount = nRows // blockSize + 1
    year_arr = gdf['year'].to_gpu_array()
    month_arr = gdf['month'].to_gpu_array()
    day_arr = gdf['day'].to_gpu_array()
    outputCol = cuda.device_array ( shape=(nRows), dtype=day_arr.dtype.name)
    
    day_of_the_week_kernel[(blockCount),(blockSize)](outputCol, year_arr, month_arr, day_arr)
    gdf.add_column(name='day_of_week', data = outputCol)
    gdf['day_of_week'] = gdf['day_of_week'].astype('float32')
    return gdf

def add_features(df):
    df['hour'] = df['pickup_datetime'].dt.hour
    df['year'] = df['pickup_datetime'].dt.year
    df['month'] = df['pickup_datetime'].dt.month
    df['day'] = df['pickup_datetime'].dt.day
    
    df = df.drop('pickup_datetime')
    
    df = day_of_week(df)
    df['is_weekend'] = (df['day_of_week']/4).floor()
    df = haversine_distance(df)
    return df

parts = [dask.delayed(add_features)(part) for part in taxi_df.to_delayed()]
taxi_df = dask_cudf.from_delayed(parts)

In [8]:
# filter data
query_frags = [
    'fare_amount > 0 and fare_amount < 500',
    'passenger_count > 0 and passenger_count < 6',
    'pickup_longitude > -75 and pickup_longitude < -73',
    'dropoff_longitude > -75 and dropoff_longitude < -73',
    'pickup_latitude > 40 and pickup_latitude < 42',
    'dropoff_latitude > 40 and dropoff_latitude < 42'
]
taxi_df = taxi_df.query(' and '.join(query_frags))

taxi_df.head().to_pandas()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,fare_amount,hour,year,month,day,day_of_week,is_weekend,h_distance
0,1,0.7,-73.994770,40.736828,1,-73.982227,40.731790,6.5,20,2013,11,9,3.0,0.0,1.195381
1,1,1.4,-73.982392,40.773382,1,-73.960449,40.763995,8.5,20,2013,11,9,3.0,0.0,2.120985
2,2,2.3,-73.988570,40.739406,1,-73.986626,40.765217,11.5,20,2013,11,9,3.0,0.0,2.872915
3,1,1.7,-73.960213,40.770464,1,-73.979863,40.777050,7.5,20,2013,11,9,3.0,0.0,1.808353
4,1,0.9,-73.995371,40.717248,1,-73.984367,40.720524,6.0,20,2013,11,9,3.0,0.0,0.995734


# Split into Training Set

In [17]:
# ToDo: use multi-column groupby to compute a more meaningful train/test split boundary
X_train = taxi_df.query('day < 25')

# we'll predict fare_amount, get it as a separate DF
def get_column_as_df(df, col):
    return df[[col]]

parts = [dask.delayed(get_column_as_df)(part, 'fare_amount') for part in X_train.to_delayed()]
Y_train = dask_cudf.from_delayed(parts)

# drop fare_amount from the training set
parts = [dask.delayed(clean)(part, dict.fromkeys(['fare_amount'])) for part in X_train.to_delayed()]
X_train = dask_cudf.from_delayed(parts)

# Train the XGBoost Regression Model

In [18]:
%%time
import dask_xgboost as dxgb_gpu

params = {
 'learning_rate': 0.05,
  'max_depth': 8,
  'objective': 'reg:linear',
  'subsample': 0.8,
  'gamma': 1,
  'silent': True,
  'verbose_eval': True,
  'tree_method':'gpu_hist',
  'n_gpus': 1
}

bst = dxgb_gpu.train(client, params, X_train, Y_train, num_boost_round=100)

CPU times: user 4.75 s, sys: 40 ms, total: 4.79 s
Wall time: 4min 8s


## Create & Generate Predictions for Test Set

In [31]:
# ToDo: use more meaningful train/test split logic
X_test = taxi_df.query('day >= 25').repartition(npartitions=5)
parts = [dask.delayed(get_column_as_df)(part, 'fare_amount') for part in X_test.to_delayed()]
Y_test = dask_cudf.from_delayed(parts)

parts = [dask.delayed(clean)(part, dict.fromkeys(['fare_amount'])) for part in X_test.to_delayed()]
X_test = dask_cudf.from_delayed(parts)

In [32]:
# generate predictions on the test set
Y_test['prediction'] = dxgb_gpu.predict(client, bst, X_test)

# Compute Root Mean Squared Error

In [33]:
Y_test['squared_error'] = (Y_test['prediction'] - Y_test['fare_amount'])**2
Y_test.head().to_pandas()

,fare_amount,prediction,squared_error
205295,13.0,11.645109,1.835729
205431,7.5,7.328209,0.029512
205493,8.0,9.405614,1.975750
205805,8.0,7.701920,0.088852
206044,14.5,14.934761,0.189017


In [34]:
math.sqrt(Y_test.squared_error.mean().compute())

2.4004054596100395

# Appendix

In [17]:
import os
import pandas as pd

# generate list of all files
base_dir = '/data/nyc_taxi/raw/'
files = []
for year in range(2009, 2019):
    for fn in os.listdir(base_dir+str(year)):
        if 'yellow' in fn:
            files.append(base_dir+str(year)+'/'+fn)

# get list of headers
def get_columns(fn):
    df = pd.DataFrame()
    with open(fn, 'r') as fp:
        df['year'] = [fn.split('-')[-2].split('_')[-1]]
        df['month'] = [fn.split('-')[-1].split('.')[0]]
        df['line'] = [fp.readline()]
    return df

parts = [dask.delayed(get_columns)(fn) for fn in files]
res = dask.dataframe.from_delayed(parts)
res.repartition(npartitions=1).compute().line.drop_duplicates()

0    vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_...
0    vendor_id,pickup_datetime,dropoff_datetime,pas...
0    vendor_id, pickup_datetime, dropoff_datetime, ...
0    VendorID,tpep_pickup_datetime,tpep_dropoff_dat...
0    VendorID,tpep_pickup_datetime,tpep_dropoff_dat...
0    VendorID,tpep_pickup_datetime,tpep_dropoff_dat...
Name: line, dtype: object

In [6]:
# return columns in df2 but not df1
def column_delta(df1, df2):
    return list(set(df2.columns.map(str.lower)) - set(df1.columns.map(str.lower)))

In [6]:
# years >= 2015 data has different column names
# remap to match existing schema
newer_df = dask_cudf.read_csv('/data/nyc-taxi/2015/yellow_tripdata_2015-1*')

# data for 2015+ has more columns
# assume we should drop them
for col in column_delta(taxi_df, newer_df):
    col_map[col] = None

#ratecodeid and tpep_pickup_datetime map to columns we had in years < 2014
col_map['ratecodeid'] = 'rate_code'
col_map['tpep_pickup_datetime'] = 'pickup_datetime'

parts = [dask.delayed(clean_data)(part, col_map) for part in newer_df.to_delayed()]
newer_df = dask_cudf.from_delayed(parts)

taxi_df = taxi_df.append(newer_df)